# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [4]:
!pip install -qU openai

In [5]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [6]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [7]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "Mikes First Technical Assistant" # @param {type: "string"}
instructions = "Answer succintly and professionally.  Your audience is technical." # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [8]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [9]:
assistant

Assistant(id='asst_ivYpvA4YSMEWuvZwed5DVRqa', created_at=1713105685, description=None, file_ids=[], instructions='Answer succintly and professionally.  Your audience is technical.', metadata={}, model='gpt-3.5-turbo', name='Mikes First Technical Assistant', object='assistant', tools=[])

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [10]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [11]:
thread

Thread(id='thread_1lzV5JHc1RaLMCBe7Rdf6Npr', created_at=1713105711, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [12]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is the difference between functional programming and declarative programming?"
)

Again, let's examine our `message` object!

In [13]:
message

Message(id='msg_ZDUZ7kWFeZsLafVi4UYBvcXF', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the difference between functional programming and declarative programming?'), type='text')], created_at=1713105806, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_1lzV5JHc1RaLMCBe7Rdf6Npr')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [14]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "Think about the question by identifying key concepts.  Identify important related concepts." # @param {type: "string"}

Let's run our Thread!

In [15]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [16]:
run

Run(id='run_KK65i7VeStSmBF5MGOkKEs9X', assistant_id='asst_ivYpvA4YSMEWuvZwed5DVRqa', cancelled_at=None, completed_at=None, created_at=1713105878, expires_at=1713106478, failed_at=None, file_ids=[], instructions='Answer succintly and professionally.  Your audience is technical. Think about the question by identifying key concepts.  Identify important related concepts.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_1lzV5JHc1RaLMCBe7Rdf6Npr', tools=[], usage=None, temperature=1.0, max_completion_tokens=None, max_prompt_tokens=None, truncation_strategy={'type': 'auto', 'last_messages': None}, incomplete_details=None, response_format='auto', tool_choice='auto')

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [17]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [18]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [19]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [20]:
messages.data[0]

Message(id='msg_YSB20g409ebPWpIue3ld0UmA', assistant_id='asst_ivYpvA4YSMEWuvZwed5DVRqa', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Functional programming is a programming paradigm where functions are treated as first-class citizens and the focus is on evaluating expressions rather than executing commands. Declarative programming, on the other hand, is a style of programming where the programmer describes the desired result without explicitly specifying how to achieve it. Both paradigms allow for more concise, readable, and maintainable code compared to imperative programming.'), type='text')], created_at=1713105879, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_KK65i7VeStSmBF5MGOkKEs9X', status=None, thread_id='thread_1lzV5JHc1RaLMCBe7Rdf6Npr')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [ ]:
!wget https://www.gutenberg.org/files/84/84-h/84-h.htm -o frankenstein.html

Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [61]:
#Using the 2022 Kia Telluride manual (one of them anyways)
file_reference = client.files.create(
  file=open("MY22_Telluride_FFG_r7_Reference.pdf", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [62]:
file_reference

FileObject(id='file-wbcFKEJwdfIIq6ecjrM9kCu9', bytes=5522701, created_at=1713108301, filename='MY22_Telluride_FFG_r7_Reference.pdf', object='file', purpose='assistants', status='processed', status_details=None)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [63]:
assistant = client.beta.assistants.create(
  name=name + "+ Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)

In [64]:
name

'Mikes First Technical Assistant'

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [65]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How often should I change the oil?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress


Let's look at the final result!

In [66]:
messages

SyncCursorPage[Message](data=[Message(id='msg_VtU5QlhTvobD8cEsRNv12NV8', assistant_id='asst_T9H0WetYWX8yqPdjVTwLPoW4', completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=360, file_citation=FileCitation(file_id='file-wbcFKEJwdfIIq6ecjrM9kCu9', quote=''), start_index=348, text='【7:1†source】', type='file_citation')], value='The engine oil change interval for normal operating conditions is based on the use of the recommended engine specification. It is recommended to check the engine oil level regularly, and the engine oil should be replaced according to the maintenance schedule under severe operating conditions if the recommended engine oil specification is not used【7:1†source】.'), type='text')], created_at=1713108333, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_GyMIEahaI02nIMCwUWcr3Dgz', status=None, thread_id='thread_vFvrbN7WOoP8xncKrgukypHV'), Message(id='

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [39]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

NotFoundError: Error code: 404 - {'error': {'message': "No file found with id 'file-gOQHxlr0vsAYihikXeucWMfa'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [67]:
assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter Python",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [68]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [69]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [70]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [71]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_spCGB4PCLfLeEWnqIDTvxsmO'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_GizeUfA30QsTxMEsSeFl42n8', code_interpreter=CodeInterpreter(input="# Read the first few bytes of the file to determine its type\r\nwith open(file_path, 'rb') as file:\r\n    file_signature = file.read(8)\r\n\r\nfile_signature", outputs=[CodeInterpreterOutputLogs(logs="b'%PDF-1.6'", type='logs')]), type='code_interpreter')], type='tool_calls')
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_8T3PqvMMCmaZBojOHDlMvwTt'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_puMDQZgxQCVrprpfUOYT7UcX', code_interpreter=CodeInterpreter(input="# Read the first few lines of the uploaded file to determine its type\r\nfile_path = '/mnt/data/file-wbcFKEJwdfIIq6ecjrM9kCu9'\r\nwith open(file_path, 'r') as file:\r\n    first_few_lines = [next(fi

In [72]:
messages

SyncCursorPage[Message](data=[Message(id='msg_spCGB4PCLfLeEWnqIDTvxsmO', assistant_id='asst_Tg6GlZYdTHWddt9VTzgSwLAi', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The file appears to have a PDF format. It starts with the PDF file signature "%PDF-1.6". Therefore, the uploaded file is a PDF file.'), type='text')], created_at=1713108381, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_mfaEkmKJoTIazj65kTs8r9il', status=None, thread_id='thread_2ExWHP83PVFv3sSSOCioWNHl'), Message(id='msg_8T3PqvMMCmaZBojOHDlMvwTt', assistant_id='asst_Tg6GlZYdTHWddt9VTzgSwLAi', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="It seems like the file cannot be decoded as a text file using the UTF-8 encoding. We can try to identify the type of the file by examining its binary signature. Let's read the first few bytes of the file to determine its type."), type='text')], creat

In [34]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

NotFoundError: Error code: 404 - {'error': {'message': "No file found with id 'file-gOQHxlr0vsAYihikXeucWMfa'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [52]:
!pip install -qU duckduckgo_search

In [53]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [55]:
duckduckgo_search("Who was the best pitcher for the NY Yankees in 2022?")

'The 2022 New York Yankees pitching stats seen on this page include pitching stats for every player who appeared in a game during the 2022 season. Wins, losses, earned runs, innings pitched, saves and more pitching data is included for every pitcher on the 2022 New York Yankees.\nYankee Stadium. Yankee Stadium Virtual VenueYankee Stadium Reference GuideYankee Stadium Team MembersSustainability InitiativesNew York Yankees MuseumDining GuideSocial Gathering LocationsTour Yankee StadiumPlan Your Event at Yankee StadiumFootball at Yankee StadiumMLS at Yankee StadiumVisiting New YorkUpcoming Events. GMS Field.\n2022 New York Yankees Statistics. 2021 Season 2023 Season. Record: 99-63-0, Finished 1st in AL_East ... Park Factors: (Over 100 favors batters, under 100 favors pitchers.) Multi-year: Batting - 102, Pitching - 101 One-year: Batting - 102, Pitching - 99 Pythagorean W-L: 106-56, 807 Runs, 567 Runs Allowed\nERA - Earned Run Average. View 2022 New York Yankees Stats and team leaders.\nDa

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [57]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

####❓ Question

Why does the description key-value pair matter?

#### ANSWER
The agent will use the description to determine if this function should be called.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [73]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [74]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

####❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

#### ANSWER

1. Creates a new thread
2. Creates a new thread message with our query
3. Runs the thread
4. Prints status of the thread
5. Waits for the thread to requires_action (or is ready to call a function) it submits our web browser tool as an option to use
6. Prints the status of that tool to use
7. Waits for the thread to complete
8. Prints the results


In [78]:
use_assistant("is the sun or earth larger", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
Consulting Duck Duck Go...
[{'tool_call_id': 'call_0e3U6HPpYCML44j2USV6W4Zy', 'output': "Its equatorial diameter and its polar diameter differ by only 6.2 miles (10 km). The mean radius of the sun is 432,450 miles (696,000 kilometers), which makes its diameter about 864,938 miles (1. ...\nThe Sun has a radius of 696.340 km / 432.685 mi and a diameter of 1.39 million km / 864.000 mi. Earth, for comparison, has a radius of only 2.439 km / 1.516 mi, and a diameter of just 12.742 km / 7.917 mi. All the planets in our Solar System combined account for just 0.2% of the Sun's mass.\nSun Observational Parameters Apparent diameter from Earth At 1 A.U.(seconds of arc) 1919. Maximum (seconds of arc) 1952. Minimum (seconds of arc) 1887. Distance from Earth Mean (10 6 km) 149.6 Minimum (10 6 km) 147.1 Maximum (10 6 km) 152.1\nFrom our vantage point on 

'thread_lNaOgyLp3BKCZBgszGqhhy10'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [85]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [86]:
use_assistant("IS the sun or moon larger?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_o4fNbuSDNlV0L2W5zxIkh7fd', 'output': 'The sun and the moon are not the sam. e size. The sun and the moon look the same size in the sky because they are of equal size and distance from Earth. This is untrue. We know from years of observation that the sun is about 400 times larger in diameter than the moon, and also happens to be about 400 times further away.\nThe sun and moon are different sizes and located at different distances from the Earth. ... "They say the sun is 400 times larger than the moon and that it only appears to be the same size as the ...\nJune 26, 2013. At this particular moment in Earth\'s history - although t

'thread_LWkhEfycIgHe1WOPKSVd5oiJ'

In [82]:
use_assistant("What brand is the automaker of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: The automaker of the supplied file is Kia.
user: What brand is the automaker of the supplied file?


'thread_dC9Lnb86wCfN86ggGky0VWT2'

In [83]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The size of the provided file is 5,522,701 bytes.
assistant: The provided file is a Features & Functions Guide for a Kia vehicle. The content appears to be related to the operation and features of the vehicle, but no specific file size information is available within the visible text. Let's check the size of the file to determine its byte count.
user: How many bytes is the provided file?


'thread_fPwQujlaNENDRNFdMPwrMZhX'

####❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?


#### ANSWER
The LLM is deciding which tool to use based upon the query and description of the functions.


### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [87]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [89]:
query = "Give me a list of Kia telluride recalls for the last 4 years?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_Hi4QfuHZD3HF814z6fD1IJid', 'output': '23V298000. Kia America, Inc. (Kia) is recalling certain 2023 Sportage, Sportage Hybrid, Sportage Plug-in Hybrid, Niro Hybrid, Niro Plug-in Hybrid, Soul, and Telluride vehicles equipped with a ...\nThe recall covers a wide spread of Tellurides, including all 2020 through 2023 and select 2024 model Telluride vehicles manufactured from January 9, 2019, through October 19, 2023.\nJames Limbach, Reporter. • Apr 11, 2024. Kia America is recalling 427,407 model year 2020-2024 Telluride SUVs. The intermediate shaft and right front driveshaft may not be engaged fully, which ...\nKia has recalled 427,407 of its Telluride SUVs because they can roll away while in park. All Telluride vehicles made between 2020 and 2023 and certa

Now we can observe JSON mode in action!

In [90]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [91]:
completed_flag

{'completed': True}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [ ]:
### YOUR CODE HERE

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [ ]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)